## Prediction Model

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import gensim
import os
import re
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU, LSTM, Bidirectional
from tensorflow.keras.layers import Embedding
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

In [3]:
def clean_text(text):
    text = text.lower()

    pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    text = pattern.sub('', text)
    text = " ".join(filter(lambda x:x[0]!='@', text.split()))
    emoji = re.compile("["
                           u"\U0001F600-\U0001FFFF"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)

    text = emoji.sub(r'', text)
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"don't", "do not", text)
    text = re.sub(r"did't", "did not", text)
    text = re.sub(r"can't", "can not", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"couldn't", "could not", text)
    text = re.sub(r"have't", "have not", text)
    text = re.sub(r"[,.\"\'!@#$%^&*(){}?/;`~:<>+=-]", "", text)
    return text

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data_1 = pd.read_json("/content/drive/MyDrive/Sarcasm detection/Sarcasm_Headlines_Dataset.json", lines=True)
data_2 = pd.read_json("/content/drive/MyDrive/Sarcasm detection/Sarcasm_Headlines_Dataset_v2.json", lines=True)

data =  pd.concat([data_1, data_2])

In [6]:
import nltk
nltk.data.path.append('/content/drive/MyDrive/nltk_data')

In [7]:
import nltk
nltk.download('punkt')
nltk.download('words')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def CleanTokenize(df):
    head_lines = list()
    lines = df["headline"].values.tolist()

    for line in lines:
        line = clean_text(line)
        # tokenize the text
        tokens = word_tokenize(line)
        # remove puntuations
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        # remove non alphabetic characters
        words = [word for word in stripped if word.isalpha()]
        stop_words = set(stopwords.words("english"))
        # remove stop words
        words = [w for w in words if not w in stop_words]
        head_lines.append(words)
    return head_lines

head_lines = CleanTokenize(data)

In [9]:
validation_split = 0.2
max_length = 25


tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(head_lines)
sequences = tokenizer_obj.texts_to_sequences(head_lines)

word_index = tokenizer_obj.word_index
print("unique tokens - ",len(word_index))
vocab_size = len(tokenizer_obj.word_index) + 1
print('vocab size -', vocab_size)

lines_pad = pad_sequences(sequences, maxlen=max_length, padding='post')
sentiment =  data['is_sarcastic'].values

indices = np.arange(lines_pad.shape[0])
np.random.shuffle(indices)
lines_pad = lines_pad[indices]
sentiment = sentiment[indices]

num_validation_samples = int(validation_split * lines_pad.shape[0])

X_train_pad = lines_pad[:-num_validation_samples]
y_train = sentiment[:-num_validation_samples]
X_test_pad = lines_pad[-num_validation_samples:]
y_test = sentiment[-num_validation_samples:]

unique tokens -  28657
vocab size - 28658


In [10]:

import tensorflow as tf
loaded_model = tf.keras.models.load_model('/content/drive/MyDrive/Sarcasm detection/model.h5')


In [11]:
import pandas as pd
def predict_sarcasm(s):
    x_final = pd.DataFrame({"headline":[s]})
    test_lines = CleanTokenize(x_final)
    test_sequences = tokenizer_obj.texts_to_sequences(test_lines)
    test_review_pad = pad_sequences(test_sequences, maxlen=max_length, padding='post')
    pred = loaded_model.predict(test_review_pad)
    pred*=100
    if pred[0][0]>=50: return "It's a sarcasm!"
    else: return "It's not a sarcasm."

# Make predictions using the loaded model
prediction = predict_sarcasm("I just won million dollars")
print(prediction)


1/1 [==============================] - 0s 423ms/step
It's a sarcasm!


In [12]:
predict_sarcasm("I want million dollars.")

1/1 [==============================] - 0s 26ms/step


"It's not a sarcasm."

##Getting Data for Analysis

In [13]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

wb = gc.open_by_key("1A5b0qRBglWZJVczNz5dhO4Rdreent-nQCeQ4C_rqtP0")

ws = wb.worksheet('Sheet1')

# get_all_values gives a list of rows.
rows = ws.get_all_values()

In [14]:
rows = pd.DataFrame(rows,columns=['username','FullName','likes','comments','Date','description','type','hashtag','location','polarity'])

In [15]:
import pandas as pd

In [16]:
def predict_sarcasm(df):
    x_final = pd.DataFrame({"headline": df['description']})
    test_lines = CleanTokenize(x_final)
    test_sequences = tokenizer_obj.texts_to_sequences(test_lines)
    test_review_pad = pad_sequences(test_sequences, maxlen=max_length, padding='post')
    preds = loaded_model.predict(test_review_pad)
    sarcasm_scores = preds[:, 0] * 100
    sarcasm_labels = ['Negative' if score >= 63 else 'It\'s not a sarcasm.' for score in sarcasm_scores]
    sarcasm_codes = [0 if label == 'Negative' else 1 for label in sarcasm_labels]
    output_df = pd.DataFrame({'Sentence': df['description'], 'Sentiment': sarcasm_labels, 'Score': sarcasm_codes})
    output_df = pd.concat([df, output_df['Sentiment'], output_df['Score']], axis=1)
    return output_df

In [ ]:
output_df = predict_sarcasm(rows)

In [ ]:
df = output_df['description']
df = pd.DataFrame({'description':df})

In [ ]:
df.drop

In [ ]:
sarcasm_df = output_df[output_df['Score'] == 0]
print(sarcasm_df['description'])

In [ ]:
non_sarcasm_df = output_df[output_df['Score']==1]
df = non_sarcasm_df['description']
df = pd.DataFrame({'description':df})

## Defining the library of Positive, Negative and Connecting Words

In [ ]:
#defining positive, negative and connectors lists of words

positive = ["love","like","happy","good","excellent","great","nice","wonderful","fantastic","amazing","awesome","terrific","fabulous","delightful","pleasure","enjoy",
    "satisfy","smile","laugh","success","kind","generous","gracious","forgiving","compassionate","understanding","trust","loyal","honest","fair","brave",
    "adventurous","confident","creative","smart","intelligent","insightful","inspiring","motivated","passionate","productive","focused","disciplined","healthy",
    "energetic","athletic","beautiful","attractive","fashionable","stylish","successful","wealthy","rich","prosperous","abundant","blessed","fortunate","lucky",
    "grateful","content","peaceful","calm","relaxed","tranquil","serene","happy","joyful","blissful","ecstatic","hopeful","optimistic","courageous","confident",
    "proud","admire","respect","appreciate","significant","glad","perfect","rapturous", "fascinating", "enriching", "tranquilizing", "blissful", "serendipitous", "inspirational", "pleasurable", "upbeat", "ecstatic",
    "delicious", "exhilarating", "glamorous", "majestic", "stunning", "irresistible", "breathtaking", "prodigious", "miraculous", "miracle","thrilling",
    "mind-blowing", "exquisite", "alluring", "extraordinary", "spectacular", "outstanding", "remarkable", "unforgettable", "marvelous", "unbeatable", "brilliant", "genius", "stellar", "fantabulous", "incredible", "sublime",
    "superb", "terrific", "top-notch", "wondrous",""]

negative = ["not","neither","don't","didn't","wouldn't","shouldn't","haven't","hadn't","can't","hate","unhappy","bad","terrible","uncomfortable","horrible","awful","disgusting","ugly","painful",
    "sorrow","hurt","upset","frustrated","angry","annoyed","disappointed","regret","guilt","shame","fear","scared","anxious","worried","nervous","doubt","insecure",
    "jealous","envy","betray","abandon","alone","lonely","depressed","sad","miserable","gloomy","hopeless","despair","powerless","worthless","rejected","humiliated",
    "insulted","offended","disrespected","unappreciated","frustrated","overwhelmed","exhausted","tired","bored","stressed","irritated","annoying","negative","critical",
    "mean","cruel","selfish","arrogant","ignorant","stupid","lazy","unmotivated","unproductive","unsuccessful","poor","struggle","failure","loss","difficult","tragic",
    "disastrous","devastating","pathetic","disappointing","regretful","heartbroken","wounded","insensitive","impolite","unfriendly","cynical","pessimistic","cowardly",
    "indecisive","irresponsible","foolish","weak","worthless","incompetent","dependent","unreliable","untrustworthy","unhealthy","unfit","unattractive","poor","broke",
    "struggling","difficult","disgusted","offended","trapped","stuck","lonely","empty","lost","confused","ashamed","humiliated","horrified","disillusioned","depressed",
    "unhappy","bitter","enraged","resentful","discontented","displeased","worried","nervous","fearful","anxious","terrified","petrified","timid","shy","insecure",
    "uncertain","doubtful","skeptical","disbelieving","inattentive","distracted","absent-minded","forgetful","neglectful","careless","reckless","irresponsible",
    "impulsive","careless","inconsiderate","thoughtless","selfish","self-centered","egotistical","arrogant","ignorant","inflexible","stubborn","rigid","boring","weird"
    "monotonous","repetitive","tedious","dull","uninteresting","unexciting","mundane","trivial","unimportant","insignificant","meaningless","worthless","disappointed"]

#connectors are the subordinating conjunctions which join two or more phrases + punctuations
connectors = ["for","and","nor","but","or","yet","so",",",";"]

## Function to encode the sentence

In [ ]:
def encodeSentence(sentence):
  # words = sentence.split()
  words = re.findall(r"[\w']+|[^\w\s]", sentence)
  encoded_sentence = []
  for word in words:
    if any(char.isdigit() or char.isalpha() for char in word):
      if word in positive:
        encoded_sentence.append(2)

      elif word in negative:
        encoded_sentence.append(0)

      elif word in connectors:
        encoded_sentence.append(3)

      else:
        encoded_sentence.append(1)

    else:
      encoded_sentence.append(3)

  return encoded_sentence

## Function to convert the encoded sentence into a tree

In [ ]:
def createTree(encoded_sentence):
  tree_diagram =[]
  N = len(encoded_sentence)
  count = 0

  for i in range(0,N+1):
    tree_diagram.append([])

  for code in encoded_sentence:
    if count == N:
      tree_diagram[count-1].append(code)
    elif count == 0:
      tree_diagram[count].append(code)
      tree_diagram[count].append(None)
    else:
      tree_diagram[count].append(code)
      tree_diagram[count].append(None)

    count = count+1

  return tree_diagram

## Function to compute the values for sentiment tree

In [ ]:
def calculateValue(temp,tree):
  result = 1

  if tree[temp][0]==0 and tree[temp][1]==1:
        result=0
  elif tree[temp][0] == 0 and tree[temp][1] == 0:
        result = 2
  elif tree[temp][0] == 1 and tree[temp][1] == 1:
        result = 1
  elif tree[temp][0] == 1 and tree[temp][1] == 0:
        result = 0
  elif tree[temp][0] == 1 and tree[temp][1] == 2:
        result = 2
  elif tree[temp][0] == 2 and tree[temp][1] == 2:
        result = 2
  elif tree[temp][0] == 2 and tree[temp][1] == 1:
      result = 2
  elif tree[temp][0] == 0 and tree[temp][1] == 2:
        result = 0
  elif tree[temp][0] == 2 and tree[temp][1] == 0:
        result = 0

  return result

## Function to calculate the Sentiment from sentiment tree

In [ ]:
def calculateSentiment(temp):
  result = 1

  if sentiment_tree[temp][0]==0 and sentiment_tree[temp][1]==1:
        result=0
  elif sentiment_tree[temp][0] == 0 and sentiment_tree[temp][1] == 0:
        result = 0
  elif sentiment_tree[temp][0] == 1 and sentiment_tree[temp][1] == 1:
        result = 1
  elif sentiment_tree[temp][0] == 1 and sentiment_tree[temp][1] == 0:
        result = 0
  elif sentiment_tree[temp][0] == 1 and sentiment_tree[temp][1] == 2:
        result = 2
  elif sentiment_tree[temp][0] == 2 and sentiment_tree[temp][1] == 2:
        result = 2
  elif sentiment_tree[temp][0] == 2 and sentiment_tree[temp][1] == 1:
      result = 2
  elif sentiment_tree[temp][0] == 0 and sentiment_tree[temp][1] == 2:
        result = 2
  elif sentiment_tree[temp][0] == 2 and sentiment_tree[temp][1] == 0:
        result = 0

  return result

## Getting sentences for analysis

In [ ]:
#for predefined list of sentences
testSentences = [
    "I absolutely love this toothpaste, it leaves my teeth feeling clean and fresh.",
    "This toothpaste is terrible, it made my mouth feel dry and uncomfortable.",
    "I've been using this toothpaste for a week and I haven't noticed any difference in my teeth.",
    "I'm not a fan of the taste of this toothpaste, but it does the job.",
    "I've been using this toothpaste for years and I wouldn't use anything else.",
    "This toothpaste is too expensive for what it does.",
    "I tried this toothpaste and it gave me a headache.",
    "This toothpaste has a refreshing minty flavor that I love.",
    "I bought this toothpaste based on the positive reviews, but I was disappointed.",
    "I can't stand the taste of this toothpaste, it's way too strong.",
    "I noticed a significant improvement in my teeth after using this toothpaste for a week.",
    "This toothpaste leaves a weird aftertaste in my mouth.",
    "I'm not sure if this toothpaste is making a difference, but it does taste good.",
    "This toothpaste is great for sensitive teeth, it doesn't cause any pain.",
    "I wouldn't recommend this toothpaste to anyone, it just doesn't work.",
    "I've been using this toothpaste for a month and I've noticed a big difference in the whiteness of my teeth.",
    "This toothpaste is perfect for traveling, it's small and easy to pack.",
    "I don't like the texture of this toothpaste, it's too gritty.",
    "I was hesitant to try this toothpaste, but I'm glad I did. It works really well.",
    "This toothpaste is very affordable and does a great job of cleaning my teeth."
]

# # or use this for getting sentences from google sheet

# from google.colab import auth
# auth.authenticate_user()

# import gspread
# from google.auth import default
# creds, _ = default()

# gc = gspread.authorize(creds)

# wb = gc.open_by_key('1Mlc--MW-RMXPlgC6YTbi561gcGyuehB0QcIwUJhiYW8')

# ws = wb.worksheet('Sheet1')

# # Extract all the tweets
# testSentences = ws.col_values(1)

## Prediction







##Sarcasm Model

In [ ]:
#Live Prediction
def prediction(s):
    x_final = pd.DataFrame({"headline":[s]})
    test_lines = CleanTokenize(x_final)
    test_sequences = tokenizer_obj.texts_to_sequences(test_lines)
    test_review_pad = pad_sequences(test_sequences, maxlen=max_length, padding='post')
    pred = loaded_model.predict(test_review_pad)
    pred*=100
    if pred[0][0]>=50: return "It's a sarcasm!"
    else: return "It's not a sarcasm."

In [ ]:
prediction("I was depressed. He asked me to be happy. I am not depressed anymore")

In [ ]:
prediction("I love Colgate but I dont like its taste")

##Sentiment Tree

In [ ]:
import re

result_list = ['Negative', 'Neutral', 'Positive']
final = []

testSentences = ["I love Colgate but I don't like its taste"]

for sentence in testSentences:
  # sentence = row['description']
  tree = createTree(encodeSentence(sentence))
  sentiment = []
  result = []
  encoded_sentence = encodeSentence(sentence)
  # print(sentence, encoded_sentence)
  list_splits = []
  start_index = 0
  #splitting the sentences by connectors
  for i, value in enumerate(encoded_sentence):
      if value == 3:
          list_splits.append(encoded_sentence[start_index:i])
          start_index = i + 1
  list_splits.append(encoded_sentence[start_index:])
  # print(encoded_sentence, list_splits)
  # Create a tree for each split
  for split in list_splits:
    if split:
      tree = createTree(split)
      # print(split, tree)
      #getting sentiment of splits
      N = len(split)
      temp = N
      # print(N)
      for i in range(0, N):
        temp = temp -1
        if i == 0:
          tree[temp][1]=1
        else:
          tree[temp][1] = calculateValue(temp+1,tree)
    sentiment.append(calculateValue(temp,tree))
    # print(tree, sentiment)

#       # print("Encoded Sentence:", encoded_sentence)
#       # print("Split Lists:", list_splits)
#       # print("Sentiment:", sentiment)

  #creating tree for sentiments
  sentiment_tree = createTree(sentiment)
  # print(sentiment, sentiment_tree)
  # print(len(sentiment_tree))
  N = len(sentiment)
  # print(N)
  temp = N

  for i in range(0,N):
    # print(sentiment_tree)
    temp=temp-1
    # print(temp, i)
    if i == 0:
      sentiment_tree[temp][1]=1
    else:
      sentiment_tree[temp][1] = calculateSentiment(temp+1)
  # print(sentiment, sentiment_tree)

  final.append(calculateSentiment(temp))

print(final)

## Printing the dataframe

In [ ]:
for i in final:
  if i == 0:
    print(f'{testSentences[0]} - Negative')
  if i == 1:
    print(f'{testSentences[0]} - Neutral')
  if i == 2:
    print(f'{testSentences[0]} - Positive')

#Analysis

In [ ]:
neutral_len = len(final_sentiment[final_sentiment['Sentiment'] == 'Neutral'])
positive_len = len(final_sentiment[final_sentiment['Sentiment'] == 'Positive'])
negative_len = len(final_sentiment[final_sentiment['Sentiment'] == 'Negative'])

print('Length of Neutral column:', neutral_len)
print('Length of Positive column:', positive_len)
print('Length of Negative column:', negative_len)

##Sentiment Pie Chart

In [ ]:
final_sentiment['Score'] = pd.to_numeric(final_sentiment['Score'], errors='coerce')
# Convert polarity values less than 0 to -1 and greater than 0 to 1
# df['Polarity'] = df['Polarity'].apply(lambda x: -1 if x < 0 else 1 if x > 0 else 0)

# Group the data by polarity
df_polarity = final_sentiment.groupby('Score').size().reset_index(name='counts')

# Set the labels and values for the pie chart
labels = ['Negative', 'Neutral', 'Positive']
colors = ['#FF0000','#FFFF00','#228B22']
values = df_polarity['counts'].tolist()

# Plot the pie chart
plt.pie(values, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
plt.axis('equal')
plt.show()

In [ ]:
average_sentiment = final_sentiment['Score'].mean()

average_sentiment

##Sentiment Line Graph for Each hastag

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime

# convert "Date" column to datsetime object
final_sentiment["Date"] = pd.to_datetime(final_sentiment["Date"])

# extract month from "Date" column
final_sentiment["Month"] = final_sentiment["Date"].dt.month

# group by "hashtags" and "Month" and calculate mean sentiment
grouped = final_sentiment.groupby(["hashtag", "Month"])["Score"].mean().reset_index()

# loop through each unique hashtag and plot line graph
for hashtag in grouped["hashtag"].unique():
    df = grouped[grouped["hashtag"] == hashtag]
    plt.figure()  # create a new figure for each hashtag
    plt.plot(df["Month"], df["Score"])
    plt.xlabel("Month")
    plt.xticks(range(1, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    plt.ylabel("Sentiment")
    plt.title(f"Sentiment per Month for {hashtag}")
    plt.show()  # show the plot for each hashtag


##Sentiment Distribution for each Hastag


In [ ]:
df = final_sentiment

In [ ]:
# create a dictionary of color codes
colors = {'Positive': 'green', 'Neutral': "#e6d800", 'Negative': 'red'}

# create a DataFrame with the count of each sentiment for each hashtag
df_sentiment_count = df.groupby(['hashtag', 'Sentiment'])['Sentiment'].count().unstack('Sentiment').fillna(0)

# create stacked bar chart
df_sentiment_count.plot(kind='bar', stacked=True, figsize=(10, 6), color=colors)
plt.title('Sentiment Distribution for each Hashtag')
plt.xlabel('Hashtag')
plt.ylabel('Count')
plt.show()
